# Ollama + OpenAI + Python

## 1. Odredite naziv modela

Ako ste preuzeli drugačiji model od "phi3:mini", promijenite vrijednost u ćeliji ispod. Ta varijabla će se koristiti u kodu kroz cijeli bilježnik.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Postavljanje OpenAI klijenta

Obično se OpenAI klijent koristi s OpenAI.com ili Azure OpenAI za interakciju s velikim jezičnim modelima.  
Međutim, može se koristiti i s Ollama, budući da Ollama pruža OpenAI-kompatibilno sučelje na "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generiranje odgovora u razgovoru

Sada možemo koristiti OpenAI SDK za generiranje odgovora u razgovoru. Ovaj zahtjev bi trebao generirati haiku o mačkama:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Inženjering upita

Prva poruka poslana jezičnom modelu naziva se "sistemska poruka" ili "sistemski upit", i ona postavlja opće upute za model.  
Možete pružiti vlastiti prilagođeni sistemski upit kako biste usmjerili jezični model da generira izlaz na drugačiji način.  
Izmijenite `SYSTEM_MESSAGE` dolje kako bi odgovarao kao vaš omiljeni lik iz poznatog filma/TV serije, ili pronađite inspiraciju za druge sistemske upite na [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Nakon što prilagodite sistemsku poruku, postavite prvo korisničko pitanje u `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Primjeri s nekoliko primjera

Drugi način vođenja jezičnog modela je pružanje "nekoliko primjera", niza pitanja/odgovora koji pokazuju kako bi trebao odgovarati.

Primjer u nastavku pokušava navesti jezični model da se ponaša kao asistent u nastavi, pružajući nekoliko primjera pitanja i odgovora koje bi asistent mogao dati, a zatim modelu postavlja pitanje koje bi student mogao postaviti.

Isprobajte prvo, a zatim izmijenite `SYSTEM_MESSAGE`, `EXAMPLES` i `USER_MESSAGE` za novi scenarij.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Dohvaćanje uz potpomognutu generaciju

RAG (Dohvaćanje uz potpomognutu generaciju) je tehnika kojom se jezični model potiče da točno odgovori na pitanja unutar određenog područja, tako da prvo dohvaća relevantne informacije iz izvora znanja, a zatim generira odgovor temeljen na tim informacijama.

Dostavili smo lokalnu CSV datoteku s podacima o hibridnim automobilima. Kod ispod čita CSV datoteku, pretražuje podudarnosti s korisničkim pitanjem i zatim generira odgovor temeljen na pronađenim informacijama. Imajte na umu da će ovo trajati dulje nego bilo koji od prethodnih primjera, jer se modelu šalje više podataka. Ako primijetite da odgovor i dalje nije utemeljen na podacima, možete pokušati s inženjeringom sustava ili isprobati druge modele. Općenito, RAG je učinkovitiji s većim modelima ili s fino podešenim verzijama SLM-ova.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
